In [11]:
import fasttext

"""
The training file should be this format:

__label__pp this is my text
__label__psoe this is also my text
"""

ModuleNotFoundError: No module named 'fasttext'

In [9]:
import pandas as pd
import numpy as np
import csv
data = pd.DataFrame(columns=('tweet', 'party'))
data.loc[len(data)] = ["Viva españa y el buen vino", "__label__pp"]
data.loc[len(data)] = ["Viva franco y 'fuera' los negros", "__label__vox"]
#Create training file.
data[data.columns.values[::-1]].to_csv('out.train', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [10]:
# Load input training set.
model = fasttext.train_supervised(input="out.train")

NameError: name 'fasttext' is not defined

In [ ]:
#To predict one instance.
model.predict("Why not put knives in the dishwasher?")

In [ ]:
#To test the test set. (number of samples, precision, recall)
model.test("cooking.valid")